In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

In [31]:
# X = pd.read_csv('https://storage.googleapis.com/kaggle-competitions-data/kaggle/10211/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561901199&Signature=Tok2i%2Fddx3XDpkbe%2FHOcxxVSAqGPMYC%2B1IJZMzuO6yH4BFQDUFe5zk9daZ0iQPJXBdg4TodP4eV9Z2Ycu3cPCJ%2FmGXjYlZDBwwRSZOh9oprBi4CKR5EavLNgk7BZ7Z6PgTxaPOcm3nGA6PzoFR1wI0rZOZPgl5rADNo3CVvjomaL9VK%2BQF9PCk6wsKNplnov6xbPGIiO9ZD89TCtMEG%2F0s6jixCgNysCFnfpf2IBYA3fIKtUdPyyQxWoselztESRuKkAVp20gUgSZgYG5OZnLG9X1FnWHJlm2lr48wclflCsifWkXItOxv%2FA4UgeUhxVd2D75zweqMa5tmrE4JXLtQ%3D%3D', index_col='Id')
# X_test_full = pd.read_csv('https://storage.googleapis.com/kaggle-competitions-data/kaggle/10211/test.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561901250&Signature=VV82UXmJARoOBlo4OZ4Cxag%2F9nFeOKvz2VCkISsDHvsjEkuun5FLDEPiIdBhLorM0yymalzb0cE%2FKrvQgTEHu0kWelUwmXHzso6AeyNYErmlKVrQll7G3ySY6ghWOsx6TGgY20XKPBj0hb%2FPtKMt4AqaErpKboQofb9PQgDusUoudonRXhosfr8i1VEXV9a3wLyVRuMxnIhKveStr0Y87oNO9UJOOLYfl9ZXYjP7RR2Doj1CWbYgMoC3cFdGbwlZatBKVE95VLL%2B3v80qYD4BRVt9%2FMuO929z%2FEychB7ZZeb%2Byc02zhriAGN1mBmNxy3rBdWv01nIxmy4tYM%2F5nQYQ%3D%3D', index_col='Id')
X = pd.read_csv('D:\\Learning_IT\\Data_analyses-Kaggle\\Data_House_Pricing\\train.csv', index_col='Id')
X_test_full = pd.read_csv('D:\\Learning_IT\\Data_analyses-Kaggle\\Data_House_Pricing\\test.csv', index_col='Id')

In [3]:
# Separating target value and rest of data in train data set
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

In [4]:
# Calculating number of NA in each column
missing_val_count_by_column = (X.isnull().sum())
missing_val_count_by_column[missing_val_count_by_column > 0]

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [5]:
# So, a lot of variables are missing in columns: Alley, PoolQC, Fence, MiscFeature (more than half) - 
# so I decided to exclude these vars from model prediction.
# Also it was decided to drop "Utilities" variable, because it is almost constant (except 1 value, that test-set doesn't 
# include)
X = X.drop(['Alley', 'PoolQC','Fence','MiscFeature', 'Utilities', 'FireplaceQu'], axis=1)
X_test_full = X_test_full.drop(['Alley', 'PoolQC','Fence','MiscFeature', 'Utilities', 'FireplaceQu'], axis=1)

In [6]:
# We also want to leave columns only with unique number of categorical variables less than 10, 
# so I decided to select categorical columns with relatively low cardinality

categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"]
# categorical_cols = [cname for cname in X.columns if X[cname].nunique() < 10 and 
#                         X[cname].dtype == "object"]
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]
all_cols = categorical_cols + numerical_cols
X_cor = X[all_cols].copy()
X_test = X_test_full[all_cols].copy()

**HERE WILL START PART WITH PIPELINE AND OneHotEncoder**

In [16]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_cor, y, train_size=0.7, test_size=0.3,
                                                               random_state=0)

numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

model = XGBRegressor(n_estimators = 900, learning_rate = 0.04, 
                     subsample = 0.8)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

In [17]:
clf.fit(X_train, y_train)

[00:50:54] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
     steps=[('preprocessor', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('num', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
       verbose=0), ['MSSubClass', 'LotFrontage', 'LotArea', ...ha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.8, verbosity=1))])

In [18]:
cross_val_score(clf, X_train, y_train, cv = 5).mean()

[00:51:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:51:19] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:51:31] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:51:47] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:51:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.8671408519990905

In [19]:
preds11 = clf.predict(X_valid)

# # Evaluate the model
score11 = mean_absolute_error(y_valid, preds11)
print('MAE:', score11)

MAE: 15463.418539526256


**HERE THIS PART WAS ENDED**

**HERE START PART WITH MANUAL PREPROCESSING**

In [20]:
X_cor = pd.get_dummies(X_cor)
X_test = pd.get_dummies(X_test)

si = SimpleImputer(strategy='mean')

X_cor_ed = pd.DataFrame(si.fit_transform(X_cor))
X_cor_ed.columns = X_cor.columns

X_test_ed = pd.DataFrame(si.fit_transform(X_test))
X_test_ed.columns = X_test.columns
X_test_ed.index = X_test.index

In [21]:
train_cols = X_cor_ed.columns
test_cols = X_test_ed.columns
common_cols = train_cols.intersection(test_cols)
train_not_test = train_cols.difference(test_cols)
train_not_test

Index(['Condition2_RRAe', 'Condition2_RRAn', 'Condition2_RRNn',
       'Electrical_Mix', 'Exterior1st_ImStucc', 'Exterior1st_Stone',
       'Exterior2nd_Other', 'GarageQual_Ex', 'Heating_Floor', 'Heating_OthW',
       'HouseStyle_2.5Fin', 'RoofMatl_ClyTile', 'RoofMatl_Membran',
       'RoofMatl_Metal', 'RoofMatl_Roll'],
      dtype='object')

In [22]:
# So now we will take columns that are in common between train and test datasets.
X_cor_ed = X_cor_ed.drop(train_not_test, axis=1)


In [23]:
# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X_cor_ed, y, train_size=0.7, test_size=0.3,
                                                               random_state=0)

In [24]:
model_xgb = XGBRegressor(n_estimators = 900, learning_rate = 0.04, 
                     subsample = 0.8)

model_xgb.fit(X_train, y_train)

[00:52:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.04, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=900,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=0.8, verbosity=1)

In [25]:
cross_val_score(model_xgb, X_train, y_train, cv = 5).mean()

[00:52:46] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:53:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:53:53] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:54:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[00:54:56] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.867370700336313

In [26]:
preds = model_xgb.predict(X_valid)

# # Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)


MAE: 15023.990341395547


In [28]:
## Writing output to file

preds_out = model_xgb.predict(X_test_ed)
output = pd.DataFrame({'Id': X_test_ed.index,
                       'SalePrice': preds_out})

output.to_csv('submission_1.csv', index=False)

In [ ]:
# RESULTS:

# n_estimators=800, max_depth=4, learning_rate=0.05, - MAE 16492.056493471748
# n_estimators = 900, learning_rate = 0.04, max_depth = 4, reg_alpha = 1, reg_lambda = 1, subsample = 0.5
#                                        - MAE MAE: 15750.379468107876
# n_estimators = 900, learning_rate = 0.04, subsample = 0.8 - MAE: 15617.851789918665

# imputer mean: model n_estimators = 900, learning_rate = 0.04, subsample = 0.8 - MAE: 15036.240002497147
 
# pipeline with imputer mean for num, delete FireplaceQu, without limitation for categoric variables + 
#  + XGB n_estimators = 900, learning_rate = 0.04, subsample = 0.8 - MAE: 15023.990341395547


**2-ND MODEL RANDOM FOREST**

In [27]:
model_rf = RandomForestRegressor(criterion = 'mse', max_depth = 5, min_samples_split = 4, n_estimators = 400)
model_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=4,
           min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [28]:
cross_val_score(model_rf, X_train, y_train, cv = 5).mean()

0.8246474002773507

In [29]:
preds_rf = model_rf.predict(X_valid)

# # Evaluate the model
score2 = mean_absolute_error(y_valid, preds_rf)
print('MAE:', score2)


MAE: 19669.5457386117


In [24]:
## Writing output to file

preds_out = model_rf.predict(X_test_ed)
output = pd.DataFrame({'Id': X_test_ed.index,
                       'SalePrice': preds_out})

output.to_csv('submission_2.csv', index=False)